In [42]:
using GaussianProcesses, PGFPlots, SumProductNetworks
using StatsFuns, Distributions, JLD, ProgressMeter, MultivariateStats, FileIO
import SumProductNetworks.add!

include("utilFunctions.jl")
include("dataTypes.jl")
include("dataTypeUtilFunctions.jl")
include("computationFunctions.jl")
include("regionGraph.jl")
include("regionGraphUtils.jl")
include("gpUtils.jl")

optimize2! (generic function with 1 method)

In [84]:
dataset = "ccpp"

"ccpp"

In [85]:
f = File(format"JLD","../data/uci/$(dataset)")
data = JLD.load(f);

X = Float64.(data["X"])
y = Float64.(data["y"]);

#Axis([
#        PGFPlots.Linear(X[:,1], y[:, 1], onlyMarks=true)
#    ])

In [87]:
size(X)

(9568, 4)

In [58]:
run = 1

N = size(X, 1)
Int(ceil(N * 0.8)) # 80 / 20 split

Ntrain = Int(ceil(N * 0.8))
Dy = size(y, 2)

srand(1234)
ids = collect(1:size(X, 1))
trainingIds = shuffle(ids)[1:Ntrain]
testingIds = shuffle(setdiff(ids, trainingIds));

Xtest = X[testingIds,:]
ytest = y[testingIds, :]

Xtrain = X[trainingIds, :]
ytrain = y[trainingIds, :];

# mean prediction
meanY = mean(ytrain,1)
rmse_mean = sqrt(mean((ytest .- meanY).^2))
println("Mean: ", rmse_mean)

# LLS prediction
a = llsq(Xtrain, ytrain, bias = false)

yhat = Xtest * a
rmse_lls = sqrt(mean((yhat .- ytest).^2))
println("LLS: ", rmse_lls)

# ridge regression prediction
α = 0.01
a = ridge(Xtrain, ytrain, α, bias = false)

yhat = Xtest * a
rmse_ridge = sqrt(mean((yhat .- ytest).^2))
println("Ridge: ", rmse_ridge)

Mean: 16.702849359856497
LLS: 10.74452093240198
Ridge: 10.745208852871137


In [66]:
# gaussian process regression (if N < 1000)
rmse_gp_fixed_noise = Inf
rmse_gp_opt_noise = Inf

if Ntrain < 1000
    mZero = MeanZero()
    kern = SE(-1.0,0.0) 
    logObsNoise = -1.0

    yhat_fixed = zeros(size(ytest))
    yhat_opt = zeros(size(ytest))

    for yi in 1:Dy

        # fixed noise
        gp_fixed = GP(Xtrain', vec(ytrain[:,yi]), MeanZero(), SE(-1.0,0.0), -1.)
        #optimize2!(gp_fixed, mean = false, kern = true, noise = false, lik=false, iterations = 1, lowerBound = -100, upperBound = 100)
        #set_priors!(gp_fixed.k, [Normal(-1.0,1.0),Normal(0.0,1.0)])

        #samples = mcmc(gp_fixed, nIter=100,burnin=0,thin=10);
        
        μ, σ² = predict_y(gp_fixed, Xtest')
        yhat_fixed[:,yi] = μ

        # optimized noise
        gp_opt = GP(Xtrain', vec(ytrain[:,yi]), MeanZero(), SE(-1.0,0.0), -1.)
        #optimize2!(gp_opt, mean = false, kern = true, noise = false, lik=false)

        μ, σ² = predict_y(gp_opt, Xtest');
        yhat_opt[:,yi] = μ
    end

    rmse_gp_fixed_noise = sqrt(mean((yhat_fixed .- ytest).^2))
    rmse_gp_opt_noise = sqrt(mean((yhat_opt .- ytest).^2))
end

println("GP (noise fixed): ", rmse_gp_fixed_noise)
println("GP (noise opt): ", rmse_gp_opt_noise)

GP (noise fixed): 6.469543997270663
GP (noise opt): 6.469543997270663


In [73]:
global gID = 1

numSums = 1
meanFunction = MeanZero();
kernelFunctions = [LinIso(log(5.0)), SE(-1., 0.), Mat32Iso(log(5.0), log(1.0))]

kernelPriors = []

noise = -1.;

# data range
minX = vec(minimum(X, 1)) - 0.1
maxX = vec(maximum(X, 1)) + 0.1

# split size
δ = (maxX - minX) ./ 4

# maximum depth
max_depth = 1
min_samples = 500

overlap = 0.0

(rootRegion, sumRegions, gpRegions, allPartitions) = poonDomingos_ND(δ, minX, maxX, max_depth, min_samples, Xtrain);

RegionIDs = Dict(r[2] => r[1] for r in enumerate(union(sumRegions, gpRegions)));
PartitionIDS = Dict(p[2] => p[1] + maximum(values(RegionIDs)) for p in enumerate(allPartitions));

Samples: 49
Samples: 775
Samples: 398
Samples: 426
Samples: 776
Samples: 48


In [74]:
yhat_fixed = zeros(size(ytest))
yhat_opt = zeros(size(ytest))

for yi in 1:Dy

    root_ = convertToSPN_ND(rootRegion, gpRegions, RegionIDs, PartitionIDS, Xtrain, ytrain[:,yi], meanFunction, 
                    kernelFunctions, kernelPriors, noise; overlap = overlap, do_mcmc = false)
    
    gpnodes = unique(filter(n -> isa(n, GPLeaf), SumProductNetworks.getOrderedNodes(root_)));
    #map(gnode -> optimize2!(gnode.gp, mean = false, kern = true, noise = true, lik=false), gpnodes);

    fill!(root_.prior_weights, 1. / length(root_))
    fill!(root_.posterior_weights, 1. / length(root_))

    spn_update!(root_)
    spn_posterior(root_)

    μ = predict_spn!(root_, Xtest);
    yhat_fixed[:,yi] = μ
end

In [75]:
rmse_spn_fixed_noise = sqrt(mean((yhat_fixed .- ytest).^2))

6.358368481348373